In [1]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

DATA_PATH = "/kaggle/input/fyp-dataset/feralignedck/feraligned+ck/"
TOP_EMOTIONS = ["Angry", "Fear", "Happy", "Neutral", "Sadness", "Surprise"]

TRAIN_SIZE = 0.80
NUM_CLASSES = len(TOP_EMOTIONS)

2024-03-19 14:55:16.396636: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-19 14:55:16.396765: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-19 14:55:16.546326: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Define input size
input_shape = (48, 48, 1)

In [2]:
# Define input size
input_shape = (48, 48, 1)

total_images = 0
for dir_ in os.listdir(DATA_PATH):
    if dir_ in TOP_EMOTIONS:
        count = len(os.listdir(os.path.join(DATA_PATH, dir_)))
        print(f"{dir_} has {count} number of images")
        total_images += count

print(f"\ntotal images are {total_images}")

img_arr = []
img_label = []
label_to_text = {}
label = 0

for dir_ in os.listdir(DATA_PATH):
    if dir_ in TOP_EMOTIONS:
        for f in os.listdir(os.path.join(DATA_PATH, dir_)):
            img = cv2.imread(os.path.join(DATA_PATH, dir_, f), cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, input_shape[:2])  # Resize the image
            img_arr.append(np.expand_dims(img, axis=-1))
            img_label.append(label)
        print(f"loaded {dir_} images to numpy arrays...")
        label_to_text[label] = dir_
        label += 1

img_arr = np.array(img_arr)
img_label = np.array(img_label)
img_label = OneHotEncoder().fit_transform(img_label.reshape(-1, 1)).toarray()  # Convert to dense array

Surprise has 249 number of images
Fear has 728 number of images
Angry has 938 number of images
Neutral has 1230 number of images
Sadness has 1153 number of images
Happy has 2203 number of images

total images are 6501
loaded Surprise images to numpy arrays...
loaded Fear images to numpy arrays...
loaded Angry images to numpy arrays...
loaded Neutral images to numpy arrays...
loaded Sadness images to numpy arrays...
loaded Happy images to numpy arrays...


In [3]:
# Normalize pixel values
img_arr = img_arr / 255.

X_train, X_test, y_train, y_test = train_test_split(img_arr, img_label,
                                                    shuffle=True, stratify=img_label,
                                                    train_size=TRAIN_SIZE, random_state=42)

def create_cnn_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [4]:
# Train and evaluate the model
print(f"Training model with input shape: {input_shape}")
model = create_cnn_model(input_shape, NUM_CLASSES)
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

Training model with input shape: (48, 48, 1)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 10s 50ms/step - accuracy: 0.3854 - loss: 1.5376 - val_accuracy: 0.5573 - val_loss: 1.1319
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 8s 48ms/step - accuracy: 0.5911 - loss: 1.0494 - val_accuracy: 0.6395 - val_loss: 1.0200
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 8s 50ms/step - accuracy: 0.6511 - loss: 0.9326 - val_accuracy: 0.6364 - val_loss: 0.9377
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 9s 54ms/step - accuracy: 0.6947 - loss: 0.8161 - val_accuracy: 0.6672 - val_loss: 0.8613
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 9s 48ms/step - accuracy: 0.7126 - loss: 0.7544 - val_accuracy: 0.6987 - val_loss: 0.8219
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 10s 47ms/step - accuracy: 0.7602 - loss: 0.6473 - val_accuracy: 0.7133 - val_loss: 0.8207
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 8s 48ms/step - accuracy: 0.7876 - loss: 0.5972 - val_accuracy: 0.6933 - val_loss: 0.8251
Epoch 8/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 8s 48ms/step - accuracy: 0.8102 - loss: 0.5340 - val_ac

# Define input size
input_shape = (64, 64, 1)

In [5]:
# Define input size
input_shape = (64, 64, 1)

total_images = 0
for dir_ in os.listdir(DATA_PATH):
    if dir_ in TOP_EMOTIONS:
        count = len(os.listdir(os.path.join(DATA_PATH, dir_)))
        print(f"{dir_} has {count} number of images")
        total_images += count

print(f"\ntotal images are {total_images}")

img_arr = []
img_label = []
label_to_text = {}
label = 0

for dir_ in os.listdir(DATA_PATH):
    if dir_ in TOP_EMOTIONS:
        for f in os.listdir(os.path.join(DATA_PATH, dir_)):
            img = cv2.imread(os.path.join(DATA_PATH, dir_, f), cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, input_shape[:2])  # Resize the image
            img_arr.append(np.expand_dims(img, axis=-1))
            img_label.append(label)
        print(f"loaded {dir_} images to numpy arrays...")
        label_to_text[label] = dir_
        label += 1

img_arr = np.array(img_arr)
img_label = np.array(img_label)
img_label = OneHotEncoder().fit_transform(img_label.reshape(-1, 1)).toarray()  # Convert to dense array

Surprise has 249 number of images
Fear has 728 number of images
Angry has 938 number of images
Neutral has 1230 number of images
Sadness has 1153 number of images
Happy has 2203 number of images

total images are 6501
loaded Surprise images to numpy arrays...
loaded Fear images to numpy arrays...
loaded Angry images to numpy arrays...
loaded Neutral images to numpy arrays...
loaded Sadness images to numpy arrays...
loaded Happy images to numpy arrays...


In [6]:
# Normalize pixel values
img_arr = img_arr / 255.

X_train, X_test, y_train, y_test = train_test_split(img_arr, img_label,
                                                    shuffle=True, stratify=img_label,
                                                    train_size=TRAIN_SIZE, random_state=42)

def create_cnn_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [7]:
# Train and evaluate the model
print(f"Training model with input shape: {input_shape}")
model = create_cnn_model(input_shape, NUM_CLASSES)
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

Training model with input shape: (64, 64, 1)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 15s 86ms/step - accuracy: 0.3672 - loss: 1.5544 - val_accuracy: 0.5542 - val_loss: 1.1148
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 15s 89ms/step - accuracy: 0.5892 - loss: 1.0365 - val_accuracy: 0.6418 - val_loss: 0.9461
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 20s 85ms/step - accuracy: 0.6465 - loss: 0.9060 - val_accuracy: 0.6802 - val_loss: 0.8814
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 21s 89ms/step - accuracy: 0.6984 - loss: 0.7925 - val_accuracy: 0.7071 - val_loss: 0.8290
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 22s 97ms/step - accuracy: 0.7402 - loss: 0.6982 - val_accuracy: 0.6964 - val_loss: 0.8563
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 19s 86ms/step - accuracy: 0.7808 - loss: 0.6146 - val_accuracy: 0.6902 - val_loss: 0.8666
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 14s 89ms/step - accuracy: 0.8003 - loss: 0.5459 - val_accuracy: 0.6964 - val_loss: 0.8495
Epoch 8/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 20s 85ms/step - accuracy: 0.8268 - loss: 0.4812 - 

# Define input size
input_shape = (128, 128, 1)

In [8]:
# Define input size
input_shape = (128, 128, 1)

total_images = 0
for dir_ in os.listdir(DATA_PATH):
    if dir_ in TOP_EMOTIONS:
        count = len(os.listdir(os.path.join(DATA_PATH, dir_)))
        print(f"{dir_} has {count} number of images")
        total_images += count

print(f"\ntotal images are {total_images}")

img_arr = []
img_label = []
label_to_text = {}
label = 0

for dir_ in os.listdir(DATA_PATH):
    if dir_ in TOP_EMOTIONS:
        for f in os.listdir(os.path.join(DATA_PATH, dir_)):
            img = cv2.imread(os.path.join(DATA_PATH, dir_, f), cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, input_shape[:2])  # Resize the image
            img_arr.append(np.expand_dims(img, axis=-1))
            img_label.append(label)
        print(f"loaded {dir_} images to numpy arrays...")
        label_to_text[label] = dir_
        label += 1

img_arr = np.array(img_arr)
img_label = np.array(img_label)
img_label = OneHotEncoder().fit_transform(img_label.reshape(-1, 1)).toarray()  # Convert to dense array

Surprise has 249 number of images
Fear has 728 number of images
Angry has 938 number of images
Neutral has 1230 number of images
Sadness has 1153 number of images
Happy has 2203 number of images

total images are 6501
loaded Surprise images to numpy arrays...
loaded Fear images to numpy arrays...
loaded Angry images to numpy arrays...
loaded Neutral images to numpy arrays...
loaded Sadness images to numpy arrays...
loaded Happy images to numpy arrays...


In [9]:
# Normalize pixel values
img_arr = img_arr / 255.

X_train, X_test, y_train, y_test = train_test_split(img_arr, img_label,
                                                    shuffle=True, stratify=img_label,
                                                    train_size=TRAIN_SIZE, random_state=42)

def create_cnn_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [10]:
# Train and evaluate the model
print(f"Training model with input shape: {input_shape}")
model = create_cnn_model(input_shape, NUM_CLASSES)
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

Training model with input shape: (128, 128, 1)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 66s 395ms/step - accuracy: 0.3507 - loss: 1.6058 - val_accuracy: 0.5534 - val_loss: 1.1325
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 82s 393ms/step - accuracy: 0.5883 - loss: 1.0678 - val_accuracy: 0.6103 - val_loss: 1.0510
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 82s 396ms/step - accuracy: 0.6526 - loss: 0.9057 - val_accuracy: 0.6503 - val_loss: 0.9334
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 65s 397ms/step - accuracy: 0.7154 - loss: 0.7631 - val_accuracy: 0.6726 - val_loss: 0.8916
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 65s 396ms/step - accuracy: 0.7659 - loss: 0.6366 - val_accuracy: 0.6810 - val_loss: 0.8965
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 82s 396ms/step - accuracy: 0.8053 - loss: 0.5314 - val_accuracy: 0.6672 - val_loss: 0.9949
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 81s 393ms/step - accuracy: 0.8463 - loss: 0.4392 - val_accuracy: 0.6710 - val_loss: 1.0450
Epoch 8/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 82s 397ms/step - accuracy: 0.8860 - loss: 0

# Define input size
input_shape = (256, 256, 1)

In [11]:
# Define input size
input_shape = (256, 256, 1)

total_images = 0
for dir_ in os.listdir(DATA_PATH):
    if dir_ in TOP_EMOTIONS:
        count = len(os.listdir(os.path.join(DATA_PATH, dir_)))
        print(f"{dir_} has {count} number of images")
        total_images += count

print(f"\ntotal images are {total_images}")

img_arr = []
img_label = []
label_to_text = {}
label = 0

for dir_ in os.listdir(DATA_PATH):
    if dir_ in TOP_EMOTIONS:
        for f in os.listdir(os.path.join(DATA_PATH, dir_)):
            img = cv2.imread(os.path.join(DATA_PATH, dir_, f), cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, input_shape[:2])  # Resize the image
            img_arr.append(np.expand_dims(img, axis=-1))
            img_label.append(label)
        print(f"loaded {dir_} images to numpy arrays...")
        label_to_text[label] = dir_
        label += 1

img_arr = np.array(img_arr)
img_label = np.array(img_label)
img_label = OneHotEncoder().fit_transform(img_label.reshape(-1, 1)).toarray()  # Convert to dense array

Surprise has 249 number of images
Fear has 728 number of images
Angry has 938 number of images
Neutral has 1230 number of images
Sadness has 1153 number of images
Happy has 2203 number of images

total images are 6501
loaded Surprise images to numpy arrays...
loaded Fear images to numpy arrays...
loaded Angry images to numpy arrays...
loaded Neutral images to numpy arrays...
loaded Sadness images to numpy arrays...
loaded Happy images to numpy arrays...


In [12]:
# Normalize pixel values
img_arr = img_arr / 255.

X_train, X_test, y_train, y_test = train_test_split(img_arr, img_label,
                                                    shuffle=True, stratify=img_label,
                                                    train_size=TRAIN_SIZE, random_state=42)

def create_cnn_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [13]:
# Train and evaluate the model
print(f"Training model with input shape: {input_shape}")
model = create_cnn_model(input_shape, NUM_CLASSES)
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

Training model with input shape: (256, 256, 1)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 321s 2s/step - accuracy: 0.3162 - loss: 2.4323 - val_accuracy: 0.3951 - val_loss: 1.3891
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 321s 2s/step - accuracy: 0.4826 - loss: 1.3034 - val_accuracy: 0.5473 - val_loss: 1.1819
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 324s 2s/step - accuracy: 0.5716 - loss: 1.0854 - val_accuracy: 0.5450 - val_loss: 1.1812
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 322s 2s/step - accuracy: 0.6285 - loss: 0.9695 - val_accuracy: 0.6018 - val_loss: 1.0281
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 316s 2s/step - accuracy: 0.6961 - loss: 0.8091 - val_accuracy: 0.6218 - val_loss: 1.0508
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 320s 2s/step - accuracy: 0.7183 - loss: 0.7464 - val_accuracy: 0.6311 - val_loss: 0.9958
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 322s 2s/step - accuracy: 0.7705 - loss: 0.6351 - val_accuracy: 0.6541 - val_loss: 0.9617
Epoch 8/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 315s 2s/step - accuracy: 0.8141 - loss: 0.5213 - val_accu